In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print("this is a test")

In [ ]:
import pandas as pd

In [ ]:
acs_data = pd.read_csv("../input/acs-2015-county-data/acs2015_county_data.csv")
acs_data.head()

In [ ]:
cases_data = pd.read_csv("../input/covid-cases/covid_cases.txt")
cases_data.head()

In [ ]:
acs_data.columns

In [ ]:
acs_data[acs_data['CensusId'] == 53061]

In [ ]:
cases_data[cases_data['fips'].isna()]

In [ ]:
cases_data[(cases_data['fips'].isna()) & (cases_data['county'] != 'Unknown')]

In [ ]:
df = cases_data[(cases_data['fips'].isna()) & (cases_data['county'] != 'Unknown')]
df['county'].unique()

In [ ]:
acs_data[acs_data['County'] == 'Joplin']

In [ ]:
joined_df = pd.merge(cases_data, acs_data, how = "inner", right_on='CensusId', left_on='fips')
fips_found = len(joined_df)
len(cases_data) - fips_found

In [ ]:
left_join_df = pd.merge(cases_data, acs_data, how = "left", right_on='CensusId', left_on='fips')
left_join_df[left_join_df['CensusId'].isnull()]

In [ ]:
joined_df2 = joined_df[~left_join_df['CensusId'].isnull()]

In [ ]:
joined_df2

In [ ]:
#Show the column names
joined_df2.columns.values

In [ ]:
#Join mask usage data
#preview the mask data
mask_data = pd.read_csv("../input/mask-usage/mask-use-by-county.txt")
mask_data.head()


In [ ]:
#We want a single score for each county, so let's weight never, rarely, sometimes, frequently, always as 0, 1, 2, 3, 4
mask_data['mask_score'] = mask_data['NEVER']*0 + mask_data['RARELY']*1 + mask_data['SOMETIMES'] * 2 + mask_data['FREQUENTLY'] * 3 + mask_data['ALWAYS'] * 4

In [ ]:
#As a sanity check, sort by mask score - do these make sense?
mask_and_acs = pd.merge(mask_data, acs_data, how = "left", right_on='CensusId', left_on='COUNTYFP').sort_values('mask_score')

In [ ]:
#over 500000 - these look plausible if you know these places
pd.merge(mask_data, acs_data, how = "left", right_on='CensusId', left_on='COUNTYFP').sort_values('mask_score').query('TotalPop > 500000')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
plt.scatter(mask_and_acs['mask_score'], mask_and_acs['TotalPop'])


In [ ]:
#county population is really skewed, so take logs
#mask score is correlated with county pop
plt.scatter(np.log(mask_and_acs['TotalPop'].astype(float)), mask_and_acs['mask_score'])

In [ ]:
#Let's join it all together
joined_df3 = pd.merge(joined_df2, mask_data, how = "left", right_on='COUNTYFP', left_on='CensusId')


In [ ]:
#But what we really want is the latest case data
max_date = joined_df3[['date']].max()
max_date.date

In [ ]:
latest = joined_df3[joined_df3['date'] == max_date.date]

In [ ]:
latest

In [ ]:
#Obviously cases should be correlated with total pop, so let's start there
np.corrcoef(latest.cases, latest.TotalPop)

In [ ]:
latest['case_rate'] = latest['cases']/latest['TotalPop']

In [ ]:
latest['case_rate']

In [ ]:
#Correlation between county population and case *rate* is much lower
np.corrcoef(latest.case_rate, latest.TotalPop)

In [ ]:
latest

In [ ]:
plt.scatter(np.log(latest.TotalPop), latest.case_rate)

In [ ]:
#A reasonable question: explain the "shape" of this plot.

In [ ]:
#Pick out some variables:
plt.scatter(latest.White, latest.case_rate)
np.corrcoef(latest.White, latest.case_rate)

In [ ]:
plt.scatter(latest.Income, latest.case_rate)
np.corrcoef(latest.Income, latest.case_rate)
#there's a "main" cluster and then what's going on at the bottom left?

In [ ]:
plt.scatter(latest.Poverty, latest.case_rate)
np.corrcoef(latest.Poverty, latest.case_rate)

In [ ]:
plt.scatter(latest.WorkAtHome, latest.case_rate)
np.corrcoef(latest.WorkAtHome, latest.case_rate)

In [ ]:
plt.scatter(latest.mask_score, latest.case_rate)
np.corrcoef(latest.mask_score, latest.case_rate)

In [ ]:
latest

In [ ]:
#Potentially bring in voting data by county: 
#from https://dataverse.harvard.edu/file.xhtml?fileId=4819117&version=9.0
vote_data = pd.read_csv("../input/presidential-vote-by-county-2000-to-2020/countypres_2000-2020.csv")


In [ ]:
vote_data

In [ ]:
vote_data.query('year == 2020 & (candidate == "JOSEPH R BIDEN JR" | candidate == "DONALD J TRUMP")')

In [ ]:
#there are 3000 counties, so what's going on here?
vote_data.query('year == 2020 & (candidate == "JOSEPH R BIDEN JR" | candidate == "DONALD J TRUMP")').groupby(by = ['state', 'county_name', 'county_fips', 'candidate']).sum()

In [ ]:

two_candidate_totals = vote_data.query('year == 2020 & (candidate == "JOSEPH R BIDEN JR" | candidate == "DONALD J TRUMP")').groupby(by = ['state', 'county_name', 'county_fips', 'candidate']).sum()

In [ ]:
two_candidate_totals

In [ ]:
two_candidate_totals['trump_percent'] = two_candidate_totals['candidatevotes']/two_candidate_totals['totalvotes']

In [ ]:
two_candidate_totals

In [ ]:
vote_results_2020 = two_candidate_totals.query('candidate == "DONALD J TRUMP"').reset_index()[['state', 'county_name', 'county_fips', 'trump_percent']]

In [ ]:
latest_with_vote = pd.merge(latest, vote_results_2020, how = "left", left_on='fips', right_on = 'county_fips')

In [ ]:
latest_with_vote

In [ ]:
plt.scatter(latest_with_vote.trump_percent, latest.case_rate)
latest_with_vote[['trump_percent', 'case_rate']].corr()

In [ ]:
#Do some binning: average case rate by county
#not very partisan, but a  little bit
#was it more so in past?
latest_with_vote.query('trump_percent > 0.5')[['cases', 'TotalPop']].sum()

In [ ]:
latest_with_vote.query('trump_percent < 0.5')[['cases', 'TotalPop']].sum()

In [ ]:
latest_with_vote.query('trump_percent > 0.8')[['cases', 'TotalPop']].sum()

In [ ]:
latest_with_vote.query('trump_percent < 0.2')[['cases', 'TotalPop']].sum()